In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("target_table", "f_special_inv_movement")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_STG_INF.SAP_VCDMMIM03"
)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_special_inv_movement
(
  calday DATE,
  store_id STRING,
  product_id STRING,
  movetype STRING,
  reason_code INT,
  storage_location STRING,
  movement_value FLOAT,
  movement_quantity FLOAT,
  shrinkage_val FLOAT,
  clkk FLOAT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW temp_f_special_inv_movement AS 
SELECT
    CAST(calday AS DATE) AS calday, 
    plant AS store_id, 
    material AS product_id, 
    movetype, 
    matmrea AS reason_code, 
    stor_loc AS storage_location, 
    SUM(cppvlc) AS movement_value, 
    SUM(CAST(cpquabu AS FLOAT)) AS movement_quantity, 
    SUM(IF(movetype IN ('551', '552'), cppvlc, 0)) AS shrinkage_val, 
    SUM(IF(SUBSTR(movetype, 1, 1) = '7', cppvlc, 0)) AS clkk
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmmim03
WHERE
    (movetype IN ('551', '552') OR SUBSTR(movetype, 1, 1) = '7')
    AND CALDAY IN ({CALDAY_IN_STR})
    AND segment IN ('1100', '1500', '2000')
GROUP BY calday, plant, material, movetype, matmrea, stor_loc  
""")

In [0]:
if CALDAY_IN_STR != "":
    # Get the calday list string from temp table
    calday_query = f"""
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_f_special_inv_movement
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]
    print(calday_in_str)

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(
            f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_special_inv_movement
        WHERE calday IN ({calday_in_str})
        """
        ).display()
    else:
        print("no valid calday's found to delete.")
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_special_inv_movement can't be deleted using calday's from f_special_inv_movement"
    )

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_special_inv_movement").drop("yyyy","mm","dd").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_special_inv_movement ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_f_special_inv_movement
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_special_inv_movement can't be inserted using calday's from f_special_inv_movement"
    )

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_{target_table}""")

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"